In [1]:
SANDBOX_NAME = 'fmex' # Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/all_data/"



Antes de utilizar este notebook, por favor, crea una carpeta `datasets` al mismo nivel que este notebook con los ficheros CSV `pokemon.csv`, `chipotle.tsv`, `iris.data` y `worldstats.csv`





# Organizar datos

Otra de las fases del proceso de data wrangling consiste en dar una estructura a los datos, normálmente esta fase conlleva las siguientes operaciones:

- Establecer índices, renombrar columnas.

- Ordenar valores.

- Eliminar duplicados

- Filtrar información

- Añadir/eliminar registros y/o columnas.

- Editar información

- Modificar la estructura de los datos




## Establecer índices, renombrar columnas





In [2]:
import pandas as pd
import numpy as np
from collections import OrderedDict



Tomamos como ejemplo de nuevo el dataset de pokemon

In [24]:
# Local read
#pokemon = pd.read_csv('datasets/pokemon.csv')

# Sandbox read
pokemon = spark.read.csv(DATA_PATH + 'pokemon.csv', header=True, inferSchema=True).toPandas()

pokemon.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,None,39,52,43,60,50,65,1,False




En este caso podríamos querer usar el nombre (Name) como índice de la tabla, lo haríamos mediante el método `set_index()`

In [25]:
pokemon.set_index('Name', inplace=True)
pokemon.head()

,#,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
Name,,,,,,,,,,,
Bulbasaur,1,Grass,Poison,45,49,49,65,65,45,1,False
Ivysaur,2,Grass,Poison,60,62,63,80,80,60,1,False
Venusaur,3,Grass,Poison,80,82,83,100,100,80,1,False
Mega Venusaur,4,Grass,Poison,80,100,123,122,120,80,1,False
Charmander,5,Fire,None,39,52,43,60,50,65,1,False




Podemos volver al índice numérico con `reset_index`

In [26]:
pokemon.reset_index(inplace=True)
pokemon.head()

,Name,#,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,1,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,2,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,3,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,4,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,5,Fire,None,39,52,43,60,50,65,1,False


In [27]:
%%timeit -n 10

fire=pokemon[pokemon['Type 1'] == 'Fire']

865 µs ± 163 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
s_pokemon = pokemon.set_index('Type 1').sort_index()

In [29]:
%%timeit -n 10

fire_i=s_pokemon.loc['Fire']

111 µs ± 30.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)




Para modificar el nombre de las columnas tenemos estas opciones



Pasar toda la lista de nombres, con la modificación deseada, en este caso cambiar Name a Nombre

In [30]:
pokemon.columns = ['Nombre', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk','Sp. Def', 'Speed', 'Generation', 'Legendary']

print(pokemon.columns)

Index(['Nombre', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')


In [31]:
pokemon.head()

,Nombre,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,1,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,2,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,3,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,4,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,5,Fire,None,39,52,43,60,50,65,1,False




Pasar un diccionario con las modificaciones

In [17]:
pokemon.rename(columns= {'Nombre' : 'Name'}, inplace=True)
print(pokemon.columns)

Index(['Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')


In [38]:
{x:x.lower().replace(' ', '_').replace('.', '') for x in pokemon.columns}

{'nombre': 'nombre',
 'type_1': 'type_1',
 'type_2': 'type_2',
 'total': 'total',
 'hp': 'hp',
 'attack': 'attack',
 'defense': 'defense',
 'sp_atk': 'sp_atk',
 'sp_def': 'sp_def',
 'speed': 'speed',
 'generation': 'generation',
 'legendary': 'legendary'}

In [36]:
pokemon.rename(columns={x:x.lower().replace(' ', '_').replace('.', '') for x in pokemon.columns}, inplace=True) 

In [37]:
pokemon.head()

,nombre,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,Bulbasaur,1,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,2,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,3,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,4,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,5,Fire,None,39,52,43,60,50,65,1,False




El método rename también permite modificar el nombre del índice de un registro, como vemos en el siguiente ejemplo.



Comprobamos el valor del índice de Ivysaur, que es 1

In [ ]:
pokemon.head()



Modificamos el nombre del índice 1 a 3

In [39]:
pokemon.rename(index={1:3},inplace=True)
pokemon.head()

,nombre,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,Bulbasaur,1,Grass,Poison,45,49,49,65,65,45,1,False
3,Ivysaur,2,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,3,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,4,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,5,Fire,None,39,52,43,60,50,65,1,False




## Ordenar

Ordenar de forma ascendete/descendente en función de los datos o índices que acabamos de crear, es útil para otras operaciones ya que acelera las búsquedas dentro del dataset





Establecemos el Name como índice de nuevo de cara a los siguientes ejercicios.
Vemos que el índice de la tabla(nombre del pokemon) no está ordenado alfabéticamente

In [41]:
pokemon.set_index('nombre',inplace=True)
pokemon.head()

,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
nombre,,,,,,,,,,,
Bulbasaur,1,Grass,Poison,45,49,49,65,65,45,1,False
Ivysaur,2,Grass,Poison,60,62,63,80,80,60,1,False
Venusaur,3,Grass,Poison,80,82,83,100,100,80,1,False
Mega Venusaur,4,Grass,Poison,80,100,123,122,120,80,1,False
Charmander,5,Fire,None,39,52,43,60,50,65,1,False




Aplicamos una ordenación por índice

In [42]:
pokemon.sort_index(inplace=True)
pokemon.head()

,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
nombre,,,,,,,,,,,
Abomasnow,511,Grass,Ice,90,92,75,92,85,60,4,False
Abra,69,Psychic,None,25,20,15,105,55,90,1,False
Absol,393,Dark,None,65,130,60,75,60,75,3,False
Accelgor,679,Bug,None,80,70,40,100,60,145,5,False
Aegislash Blade Forme,751,Steel,Ghost,60,150,50,150,50,60,6,False




Para aplicar el sort en ordern inverso

In [43]:
pokemon.sort_index(ascending=False).head()

,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
nombre,,,,,,,,,,,
Zygarde Half Forme,795,Dragon,Ground,108,100,121,81,95,95,6,True
Zweilous,696,Dark,Dragon,72,85,70,65,70,58,5,False
Zubat,47,Poison,Flying,40,45,35,30,40,55,1,False
Zorua,632,Dark,None,40,65,40,80,40,65,5,False
Zoroark,633,Dark,None,60,105,60,120,60,105,5,False




Igualmente podemos ordenar en funcion de los valores de las columnas



Ordenamos por order ascendente de ataque

In [44]:
pokemon.sort_values('attack').head()

,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
nombre,,,,,,,,,,,
Chansey,122,Normal,None,250,5,5,35,105,50,1,False
Happiny,489,Normal,None,100,5,5,15,65,30,4,False
Shuckle,231,Bug,Rock,20,10,230,10,230,5,2,False
Blissey,262,Normal,None,255,10,10,75,135,55,2,False
Magikarp,140,Water,None,20,10,55,15,20,80,1,False




Para aplicar un sort sobre varias columnas usando distintos tipos de orden sobre cada una de ellas.

In [45]:
pokemon.sort_values(['attack','defense'], ascending=[True,False])

,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
nombre,,,,,,,,,,,
Chansey,122,Normal,None,250,5,5,35,105,50,1,False
Happiny,489,Normal,None,100,5,5,15,65,30,4,False
Shuckle,231,Bug,Rock,20,10,230,10,230,5,2,False
Magikarp,140,Water,None,20,10,55,15,20,80,1,False
Blissey,262,Normal,None,255,10,10,75,135,55,2,False
...,...,...,...,...,...,...,...,...,...,...,...
Primal Groudon,425,Ground,Fire,100,180,160,150,90,90,3,True
Mega Rayquaza,427,Dragon,Flying,105,180,100,180,100,115,3,True
DeoxysAttack Forme,430,Psychic,None,50,180,20,180,20,150,3,True




## Eliminar duplicados

Todos dataset son susceptibles de tener registros duplicados, total o parcialmente, lo que según cada caso puede tener sentido o no. Veamos como identificarlos y tratarlos. 





Cargamos de nuevo el dataset

In [55]:
# Local read
# pokemon = pd.read_csv('datasets/pokemon.csv')

# Sandbox read
pokemon = spark.read.csv(DATA_PATH + 'pokemon.csv', header=True, inferSchema=True).toPandas()

pokemon.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,None,39,52,43,60,50,65,1,False


In [58]:
pokemon = spark.read.csv(DATA_PATH + 'pokemon.csv', header=True, inferSchema=True).toPandas()

pokemon.drop('#', axis=1, inplace=True)
pokemon.rename(columns={x:x.replace(' ', '_').replace('.', '').lower() for x in pokemon.columns}, inplace=True)

In [59]:
pokemon.head()

,name,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,None,39,52,43,60,50,65,1,False




Mediante duplicated obtenemos si existe algún registro duplicado, a nivel de todos los campos

In [60]:
pokemon.duplicated().any()

False

In [48]:
# pokemon.reset_index(inplace=True)

In [49]:
# pokemon.rename(columns={'nombre':'name'}, inplace=True)

In [61]:
pokemon.head()

,name,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,None,39,52,43,60,50,65,1,False




Podemos aplicar ese filtro a un número determinado de campos

In [62]:
print('Are there Pokemos with the same name?','\n')
print(pokemon['name'].duplicated().any(),'\n')
print('Are there several Type 1 Pokemons?','\n')
print(pokemon['type_1'].duplicated().any(),'\n')

Are there Pokemos with the same name? 

False 

Are there several Type 1 Pokemons? 

True 



In [63]:
pokemon['type_1']

0        Grass
1        Grass
2        Grass
3        Grass
4         Fire
        ...   
795       Rock
796       Rock
797    Psychic
798    Psychic
799       Fire
Name: type_1, Length: 800, dtype: object



Para eliminar los registros duplicados usamos `drop_duplicates`, que requiere el parámetro inplace=True para modificar el dataset original.



En este caso no eliminaríamos ningún registro, ya que acabamos de ver que no hay pokemons iguales

In [64]:
pokemon.drop_duplicates()

,name,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,None,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True




Si aplicamos el filtro únicamente a la columna Type 1 si que encontramos duplicados, que eliminaremos,
quedándonos únicamente con la primera ocurrencia.

In [66]:
pokemon.drop_duplicates(subset=['type_1', 'type_2'], keep='first')

,name,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
4,Charmander,Fire,None,39,52,43,60,50,65,1,False
6,Charizard,Fire,Flying,78,84,78,109,85,100,1,False
7,Mega Charizard X,Fire,Dragon,78,130,111,130,85,100,1,False
9,Squirtle,Water,None,44,48,65,50,64,43,1,False
...,...,...,...,...,...,...,...,...,...,...,...
778,Phantump,Ghost,Grass,43,70,48,50,60,38,6,False
790,Noibat,Flying,Dragon,40,30,35,45,40,55,6,False
797,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True




## Aplicar filtros

Aplicar filtros sobre los datos es muy útil de cara a conocer como se distribuyen nuestros datos y con ello realizar transformaciones sobre los mismos





Para aplicar un filtro tenemos que establecer una condición sobre los datos

In [74]:
pokemon[pokemon.attack.gt(150) & 
        pokemon.defense.eq(150) & 
        ~pokemon.legendary]

,name,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
268,Mega Tyranitar,Rock,Dark,100,164,150,95,120,71,2,False




Los operadores lógicos se resumen en esta imagen

<center><img src="img/logicaloperators.png"></center>




Se pueden aplicar varios condiciones simultáneamente

In [76]:
pokemon[pokemon.attack.gt(110) & pokemon.type_1.eq('Water')]

,name,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
107,Kingler,Water,None,55,130,115,50,50,75,1,False
140,Gyarados,Water,Flying,95,125,79,60,100,81,1,False
141,Mega Gyarados,Water,Dark,95,155,109,70,130,81,1,False
283,Mega Swampert,Water,Ground,100,150,110,95,110,70,3,False
348,Sharpedo,Water,Dark,70,120,40,95,40,95,3,False
349,Mega Sharpedo,Water,Dark,70,140,70,110,65,105,3,False
374,Crawdaunt,Water,Dark,63,120,85,90,55,55,3,False
422,Primal Kyogre,Water,None,100,150,90,180,160,90,3,True
541,Palkia,Water,Dragon,90,120,100,150,120,100,4,True


In [75]:
pokemon[(pokemon['attack'] > 110) & (pokemon['type_1'] == 'Water')]


,name,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
107,Kingler,Water,None,55,130,115,50,50,75,1,False
140,Gyarados,Water,Flying,95,125,79,60,100,81,1,False
141,Mega Gyarados,Water,Dark,95,155,109,70,130,81,1,False
283,Mega Swampert,Water,Ground,100,150,110,95,110,70,3,False
348,Sharpedo,Water,Dark,70,120,40,95,40,95,3,False
349,Mega Sharpedo,Water,Dark,70,140,70,110,65,105,3,False
374,Crawdaunt,Water,Dark,63,120,85,90,55,55,3,False
422,Primal Kyogre,Water,None,100,150,90,180,160,90,3,True
541,Palkia,Water,Dragon,90,120,100,150,120,100,4,True




Para evitar tener que agrupar varios filtros usamos .isin()

In [78]:
pokemon[pokemon.attack.between(110, 150) & 
        pokemon.type_1.isin(['Water','Grass','Fire'])]


,name,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
7,Mega Charizard X,Fire,Dragon,78,130,111,130,85,100,1,False
64,Arcanine,Fire,None,90,110,80,100,80,95,1,False
107,Kingler,Water,None,55,130,115,50,50,75,1,False
140,Gyarados,Water,Flying,95,125,79,60,100,81,1,False
147,Flareon,Fire,None,65,130,60,95,110,65,1,False
263,Entei,Fire,None,115,115,85,90,75,100,2,True
270,Ho-oh,Fire,Flying,106,130,90,110,154,90,2,True
275,Mega Sceptile,Grass,Dragon,70,110,75,145,85,145,3,False
278,Blaziken,Fire,Fighting,80,120,70,110,70,80,3,False
282,Swampert,Water,Ground,100,110,90,85,90,60,3,False




Tambien es posible usar el método `between`

In [ ]:
pokemon['Attack'].between(110,150)



Hasta ahora hemos visto como realizar filtros sobre el contenido, pero si queremos seleccionar por filas es necesario recurrir a los métodos `.iloc[], .loc[], .ix[]`

Recordamos que .iloc[] realiza la selección de registros por el índice numérico, .loc[] a través de la etiqueta del índice e .ix[] permite el uso tanto de etiquetas como de valores numéricos, sin embargo, no se recomienda su uso ya que se encuentra deprecado desde la versión 0.20.0 de Pandas.

Veamos algunos ejemplos de selección de registros con .iloc[] y .loc[]





Selección de un registro

In [79]:
pokemon.iloc[3]

name          Mega Venusaur
type_1                Grass
type_2               Poison
hp                       80
attack                  100
defense                 123
sp_atk                  122
sp_def                  120
speed                    80
generation                1
legendary             False
Name: 3, dtype: object



Selección mediante intervalo

In [80]:
pokemon.iloc[3:6]

,name,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,None,39,52,43,60,50,65,1,False
5,Charmeleon,Fire,None,58,64,58,80,65,80,1,False




Selección mediante una lista

In [82]:
pokemon.iloc[[3,7,9], [3, 2, 5]]

,hp,type_2,defense
3,80,Poison,123
7,78,Dragon,111
9,44,None,65




Selección de un registro y un único campo

In [83]:
pokemon.iloc[3, 1]

'Grass'



Selección de varios registros y varios campos

In [84]:
pokemon.iloc[[3,9,18], [1,4,5]]

,type_1,attack,defense
3,Grass,100,123
9,Water,48,65
18,Bug,90,40




Para poder usar el método .loc[] necesitamos que el dataframe tenga un índice con etiquetas

In [87]:
pokemon.reset_index(inplace=True)
pokemon.set_index('name',inplace=True)
pokemon.sort_index(inplace=True)
pokemon.head()

,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
name,,,,,,,,,,
Abomasnow,Grass,Ice,90,92,75,92,85,60,4,False
Abra,Psychic,None,25,20,15,105,55,90,1,False
Absol,Dark,None,65,130,60,75,60,75,3,False
Accelgor,Bug,None,80,70,40,100,60,145,5,False
Aegislash Blade Forme,Steel,Ghost,60,150,50,150,50,60,6,False




Selección de un registro

In [88]:
pokemon.loc['Ivysaur']

type_1         Grass
type_2        Poison
hp                60
attack            62
defense           63
sp_atk            80
sp_def            80
speed             60
generation         1
legendary      False
Name: Ivysaur, dtype: object



Selección de varios registro

In [89]:
pokemon.loc[['Ivysaur','Venusaur','Charmander']]

,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
name,,,,,,,,,,
Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
Charmander,Fire,None,39,52,43,60,50,65,1,False


In [93]:
pokemon.head()

,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
name,,,,,,,,,,
Abomasnow,Grass,Ice,90,92,75,92,85,60,4,False
Abra,Psychic,None,25,20,15,105,55,90,1,False
Absol,Dark,None,65,130,60,75,60,75,3,False
Accelgor,Bug,None,80,70,40,100,60,145,5,False
Aegislash Blade Forme,Steel,Ghost,60,150,50,150,50,60,6,False


In [102]:
#pokemon.loc['Acc',:]



Selección de varios registros y varias columnas

In [103]:
pokemon.loc[['Ivysaur','Venusaur','Charmander'], ['attack','defense']]

,attack,defense
name,,
Ivysaur,62,63
Venusaur,82,83
Charmander,52,43




## Añadir/eliminar registros y/o columnas

En ocasiones es necesario eliminar registros o columnas que no son necesarios, veamos como hacerlo.





Borrado de un único registro por etiqueta

In [104]:
print('Pokemons number','\n')
print(pokemon.shape[0],'\n')
pokemon.drop('Charmander',inplace=True)
print('Pokemons number after the deletion','\n')
print(pokemon.shape[0],'\n')

Pokemons number 

800 

Pokemons number after the deletion 

799 





Es posible eliminar una columna entera indicando su nombre y el eje correspondiente (axis=1)



Eliminamos la columna 'Generation'

In [105]:
pokemon.drop('generation',axis=1,inplace=True)
pokemon.head()

,type_1,type_2,hp,attack,defense,sp_atk,sp_def,speed,legendary
name,,,,,,,,,
Abomasnow,Grass,Ice,90,92,75,92,85,60,False
Abra,Psychic,None,25,20,15,105,55,90,False
Absol,Dark,None,65,130,60,75,60,75,False
Accelgor,Bug,None,80,70,40,100,60,145,False
Aegislash Blade Forme,Steel,Ghost,60,150,50,150,50,60,False




El borrado de columnas también es posible realizarlo mediante la built-in función `del` 

In [106]:
del pokemon['speed']
pokemon.head()

,type_1,type_2,hp,attack,defense,sp_atk,sp_def,legendary
name,,,,,,,,
Abomasnow,Grass,Ice,90,92,75,92,85,False
Abra,Psychic,None,25,20,15,105,55,False
Absol,Dark,None,65,130,60,75,60,False
Accelgor,Bug,None,80,70,40,100,60,False
Aegislash Blade Forme,Steel,Ghost,60,150,50,150,50,False




Para agregar una nueva columna haremos una asignación de un contenido a una columna que no exista



Creamos una nueva columna 'Level'

In [107]:
pokemon['level'] = pokemon['attack']*2
pokemon.head()

,type_1,type_2,hp,attack,defense,sp_atk,sp_def,legendary,level
name,,,,,,,,,
Abomasnow,Grass,Ice,90,92,75,92,85,False,184
Abra,Psychic,None,25,20,15,105,55,False,40
Absol,Dark,None,65,130,60,75,60,False,260
Accelgor,Bug,None,80,70,40,100,60,False,140
Aegislash Blade Forme,Steel,Ghost,60,150,50,150,50,False,300


In [110]:
pokemon.assign(compound_type = pokemon.type_1.fillna('') + pokemon.type_2.fillna(''))

,type_1,type_2,hp,attack,defense,sp_atk,sp_def,legendary,level,compound_type
name,,,,,,,,,,
Abomasnow,Grass,Ice,90,92,75,92,85,False,184,GrassIce
Abra,Psychic,None,25,20,15,105,55,False,40,Psychic
Absol,Dark,None,65,130,60,75,60,False,260,Dark
Accelgor,Bug,None,80,70,40,100,60,False,140,Bug
Aegislash Blade Forme,Steel,Ghost,60,150,50,150,50,False,300,SteelGhost
...,...,...,...,...,...,...,...,...,...,...
Zorua,Dark,None,40,65,40,80,40,False,130,Dark
Zubat,Poison,Flying,40,45,35,30,40,False,90,PoisonFlying
Zweilous,Dark,Dragon,72,85,70,65,70,False,170,DarkDragon




Si queremos añadir algún registro nuevo, tenemos que proporcinar la información para las columnas.



Creamos la información del nuevo pokemon

In [111]:
data = {'name': ['Charmander_BLUE'],
        'type_1': ['Fire'],
        'type_2': ['Water'],
        'hp': [100]}

new_pokemon = pd.DataFrame(data)

In [112]:
new_pokemon

,name,type_1,type_2,hp
0,Charmander_BLUE,Fire,Water,100




Añadimos el nuevo registro

In [113]:
pokemon = pokemon.append(new_pokemon, sort=True)
pokemon[pokemon['name']=='Charmander_BLUE']

,attack,defense,hp,legendary,level,name,sp_atk,sp_def,type_1,type_2
0,NaN,NaN,100,NaN,NaN,Charmander_BLUE,NaN,NaN,Fire,Water




## Editar información







Para editar el contenido del dataframe podemos modificar toda una columna en bloque, usando el método de asignación (=)

In [114]:

#pokemon['Speed'] = pokemon['Attack']/pokemon['Defense']
#pokemon['Speed'].head()
(pokemon
 .assign(speed=(pokemon
                .attack
                .div(pokemon.defense))))

,attack,defense,hp,legendary,level,name,sp_atk,sp_def,type_1,type_2,speed
Abomasnow,92.0,75.0,90,False,184.0,NaN,92.0,85.0,Grass,Ice,1.226667
Abra,20.0,15.0,25,False,40.0,NaN,105.0,55.0,Psychic,None,1.333333
Absol,130.0,60.0,65,False,260.0,NaN,75.0,60.0,Dark,None,2.166667
Accelgor,70.0,40.0,80,False,140.0,NaN,100.0,60.0,Bug,None,1.750000
Aegislash Blade Forme,150.0,50.0,60,False,300.0,NaN,150.0,50.0,Steel,Ghost,3.000000
...,...,...,...,...,...,...,...,...,...,...,...
Zubat,45.0,35.0,40,False,90.0,NaN,30.0,40.0,Poison,Flying,1.285714
Zweilous,85.0,70.0,72,False,170.0,NaN,65.0,70.0,Dark,Dragon,1.214286
Zygarde Half Forme,100.0,121.0,108,True,200.0,NaN,81.0,95.0,Dragon,Ground,0.826446
NaN,105.0,60.0,65,False,210.0,NaN,60.0,70.0,Fighting,None,1.750000




También podemos editar el contenido de un registro concreto y de alguno o todos sus campos. Para ello nos basamos en los método de selección de registros .iloc/.loc antes explicados.



Vemos la información del pokemon del índice 10

In [115]:
pokemon.iloc[10]

attack          75
defense         80
hp             165
legendary    False
level          150
name           NaN
sp_atk          40
sp_def          45
type_1       Water
type_2        None
Name: Alomomola, dtype: object



Modificamos el contenido (campos 0, 1 y 4) del registro 10 asignando nueva información.

In [116]:
pokemon.iloc[10, [0,1,4]] = [100,130,True]

In [117]:
pokemon.iloc[10]

attack         100
defense        130
hp             165
legendary    False
level         True
name           NaN
sp_atk          40
sp_def          45
type_1       Water
type_2        None
Name: Alomomola, dtype: object



## Modificar la estructura de los datos







### Pivot, stack/unstack, melt

Existen varias métodos que nos permiten transformar la estructura de las tablas, haciendo agrupaciones de datos, trasladando índices de columnas a filas y agrupando varias columnas en otras nuevas. Estos son los métodos pivot, stack/unstack y melt.






La función **pivot** se usa para crear una nueva tabla derivada a partir de otra. 



Creamos un dataframe ejemplo

In [118]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
d = pd.DataFrame(table)

In [119]:
d

,Item,CType,USD,EU
0,Item0,Gold,1$,1€
1,Item0,Bronze,2$,2€
2,Item1,Gold,3$,3€
3,Item1,Silver,4$,4€




Creamos la pivot-table

In [120]:
p = d.pivot(index='Item', columns='CType', values='USD')
p

CType,Bronze,Gold,Silver
Item,,,
Item0,2$,1$,NaN
Item1,NaN,3$,4$




Hacer **stack** en un Dataframe significa mover la columna más interna (rotando el dataframe) para convertirla en el índice de fila más interno.



Generamos un dataframe con varios índices tanto a nivel de columna como de fila

In [121]:
row_idx_arr = list(zip(['r0', 'r0'], ['r-00', 'r-01']))
row_idx = pd.MultiIndex.from_tuples(row_idx_arr)


col_idx_arr = list(zip(['c0', 'c0', 'c1'], ['c-00', 'c-01', 'c-10']))
col_idx = pd.MultiIndex.from_tuples(col_idx_arr)


d = pd.DataFrame(np.arange(6).reshape(2,3), index=row_idx, columns=col_idx)
d = d.applymap(lambda x: (x // 3, x % 3))
d

c0              c1
           c-00    c-01    c-10
r0 r-00  (0, 0)  (0, 1)  (0, 2)
   r-01  (1, 0)  (1, 1)  (1, 2)



Al hacer stack del DF estamos moviendo la columna más interior hasta el índice de las filas

In [122]:
s = d.stack()
s


c0      c1
r0 r-00 c-00  (0, 0)     NaN
        c-01  (0, 1)     NaN
        c-10     NaN  (0, 2)
   r-01 c-00  (1, 0)     NaN
        c-01  (1, 1)     NaN
        c-10     NaN  (1, 2)



Podemos deshacer la operación mediante unstack

In [123]:
u = s.unstack()
u

c0                c1             
           c-00    c-01 c-10 c-00 c-01    c-10
r0 r-00  (0, 0)  (0, 1)  NaN  NaN  NaN  (0, 2)
   r-01  (1, 0)  (1, 1)  NaN  NaN  NaN  (1, 2)



Por último tenemos la operación **melt**, que nos permite realizar la agrupación de varias columnas relacionadas en una nueva.



Creamos un dataframe

In [124]:
data = {'weekday': ["Monday", "Tuesday", "Wednesday", 
         "Thursday", "Friday", "Saturday", "Sunday"],
        'Person 1': [12, 6, 5, 8, 11, 6, 4],
        'Person 2': [10, 6, 11, 5, 8, 9, 12],
        'Person 3': [8, 5, 7, 3, 7, 11, 15]}
df = pd.DataFrame(data, columns=['weekday',
        'Person 1', 'Person 2', 'Person 3'])

In [125]:
df

,weekday,Person 1,Person 2,Person 3
0,Monday,12,10,8
1,Tuesday,6,6,5
2,Wednesday,5,11,7
3,Thursday,8,5,3
4,Friday,11,8,7
5,Saturday,6,9,11
6,Sunday,4,12,15




Realizamos la agrupación por día de la semana y por la nueva variable persona.

In [126]:
melted = pd.melt(df, id_vars=["weekday"], 
                 var_name="Person", value_name="Score")

In [127]:
melted

,weekday,Person,Score
0,Monday,Person 1,12
1,Tuesday,Person 1,6
2,Wednesday,Person 1,5
3,Thursday,Person 1,8
4,Friday,Person 1,11
5,Saturday,Person 1,6
6,Sunday,Person 1,4
7,Monday,Person 2,10
8,Tuesday,Person 2,6
9,Wednesday,Person 2,11




# Ejercicios



## Ejercicio 1 

Dado el fichero de datos datasets/chipotle.tsv

- a. Cargue el fichero en un dataframe y estudie las estructura de los datos
- b. Transforme la columna item_price a float. [x]
- c. Crear la columna unitary price (quantity e item_price) [x]
- d. Cuantos productos cuestan más de 10$ [x]
- e. ¿Cuantas veces se ha ordenado el producto mas caro? [x]
- f. ¿Cuantas veces los cientes consumen más de una Canned Soda?
- g. OPTIONAL: Cuantos articulos en promedio se ordenan en el top decil de cuentas mas caras. 




##### Solución Ejercicio 1



Cargue el fichero en un dataframe y estudie las estructura de los datos

In [128]:
data = spark.read.csv(DATA_PATH + 'chipotle.tsv', sep='\t', inferSchema=True, header=True).toPandas()
data.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NULL,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [136]:
data = (data
 .assign(item_price=(data
                     .item_price
                     .str
                     .replace('$', '').astype(float)))
)
data = data.assign(unit_price=data.item_price.div(data.quantity))
data.head()

,order_id,quantity,item_name,choice_description,item_price,unit_price
0,1,1,Chips and Fresh Tomato Salsa,NULL,2.39,2.39
1,1,1,Izze,[Clementine],3.39,3.39
2,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,8.49


In [145]:
data.item_name[data.unit_price.gt(10)].unique()

array(['Chicken Bowl', 'Steak Burrito', 'Chicken Burrito',
       'Barbacoa Bowl', 'Veggie Burrito', 'Veggie Bowl',
       'Chicken Soft Tacos', 'Steak Bowl', 'Carnitas Burrito',
       'Carnitas Bowl', 'Barbacoa Burrito', 'Chicken Salad Bowl',
       'Barbacoa Crispy Tacos', 'Veggie Salad Bowl', 'Chicken Salad',
       'Steak Salad Bowl', 'Chicken Crispy Tacos', 'Veggie Soft Tacos',
       'Barbacoa Soft Tacos', 'Carnitas Crispy Tacos',
       'Carnitas Salad Bowl', 'Barbacoa Salad Bowl', 'Steak Soft Tacos',
       'Carnitas Soft Tacos', 'Steak Crispy Tacos'], dtype=object)

In [162]:
data.set_index('item_name').unit_price.idxmax()

'Steak Salad Bowl'

In [148]:
data.item_name.eq(data.set_index('item_name').unit_price.idxmax()).sum()

29

In [169]:
(data
 [data.unit_price.eq(data.unit_price.max())]
 .groupby('item_name')
 .size()
 .sort_values(ascending=False)
 .head())

item_name
Steak Salad Bowl       19
Barbacoa Salad Bowl     5
Carnitas Salad Bowl     4
dtype: int64



Transforme la columna item_price a float

In [187]:
item_order_agg = data.groupby(['order_id', 'item_name']).quantity.sum().reset_index()
item_order_agg[item_order_agg.item_name.eq('Canned Soda') &
               item_order_agg.quantity.gt(1)].sort_values('order_id').shape[0]

24



Cuantos productos cuestan más de 10$

In [186]:
data[data.order_id == 787]

,order_id,quantity,item_name,choice_description,item_price,unit_price
1943,787,1,Steak Bowl,"[[Fresh Tomato Salsa (Mild), Tomatillo-Green C...",8.99,8.99
1944,787,2,Canned Soda,[Dr. Pepper],2.18,1.09
1945,787,1,Canned Soda,[Coca Cola],1.09,1.09
1946,787,1,Steak Burrito,"[[Tomatillo-Green Chili Salsa (Medium), Roaste...",8.99,8.99
1947,787,1,Carnitas Burrito,"[[Fresh Tomato Salsa (Mild), Roasted Chili Cor...",8.99,8.99




¿Cual es la cantidad del producto más caro ordenado?

In [ ]:
# Cuantos articulos en promedio se ordenan en el top decil de cuentas mas caras
# NUM -> DECIL

In [191]:
n_quantiles = np.linspace(0, 1, 11)
labels = [f'd{x}' for x in range(1, 11)]

['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10']

In [205]:
grouped_data = data.groupby('order_id').item_price.sum()
top_decil = grouped_data.index[pd.qcut(grouped_data, n_quantiles, labels).eq('d10')]

In [209]:
np.mean(data.loc[list(top_decil)].groupby('order_id').quantity.sum())

1.18562874251497



¿Cuantas veces los cientes consumen más de una Canned Soda?

In [ ]:
# Respuesta aqui



## Ejercicio 2 

Dado el fichero de datos datasets/iris.data



- a. Cargue el fichero en un dataframe y estudie las estructura de los datos
- b. Asigne a las columnas los siguientes nombres: sepal_length, sepal_width, petal_length, petal_width y class
- c. Elimine la columna class
- d. Asigne valores nulos a las 3 primeras filas y después elimíne las filas con NaNs
- e. Reinicie el índice de los registros




Cargue el fichero en un dataframe y estudie las estructura de los datos

In [ ]:
# Respuesta aqui



Asigne a las columnas los siguientes nombres: sepal_length, sepal_width, petal_length, petal_width y class

In [ ]:
# Respuesta aqui



Elimine la columna class

In [ ]:
# Respuesta aqui



Asigne valores nulos a las 3 primeras filas y después elimíne las filas con NaNs

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui



Reinicie el índice de los registros

In [ ]:
# Respuesta aqui



## Ejercicio 3 

Dado el dataset /datasets/worldstats.csv



- a. Cargue el fichero en un dataframe y estudie las estructura de los datos.
- b. Reasigne el índice  de modo que el DF resultante tenga el país y el año como índice de los registros
- c. Modifique la estructura del DF de modo que por cada registro haya un único valor para Population y GDP




Cargue el fichero en un dataframe y estudie las estructura de los datos.

In [ ]:
# Respuesta aqui



Reasigne el índice  de modo que el DF resultante tenga el país y el año como índice de los registros

In [ ]:
# Respuesta aqui



Modifique la estructura del DF de modo que por cada registro haya un único valor para Population y GDP

In [ ]:
# Respuesta aqui